plots for the tissue specificity

author:@rayezh

In [1]:
import pandas as pd
import numpy as np
import itertools
from scipy.stats import kruskal, wilcoxon, mannwhitneyu
from scikit_posthocs import posthoc_dunn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr, spearmanr

In [55]:
# plot the drug responses characteristics
# load training dataset: drug response dataset
# import data from combination
df= pd.read_csv('../feature/data/responses_all.tsv', sep = '\t')
df['.identifier_sample_name'] = df['.identifier_sample_name'].str.replace('ONCOLEAD_CELL_', '')
newdf = df.drop_duplicates(subset = ['dataset', '.identifier_sample_name', '.metadata_cancer_type'],keep = 'first').reset_index(drop = True)

# cell line to dataset and tissue
global c2d # cell line in each dataset
c2d = {r['.identifier_sample_name']:r['dataset'] for _,r in newdf.iterrows()} 
global c2t # cell lines in each cancer type
c2t = {r['.identifier_sample_name']:r['.metadata_cancer_type'] for _,r in newdf.iterrows()} 


# distribution of DDR efficcacy and synergies in differernt tissues

# Kruskal-Wallis Test
# Dunn's test

In [56]:
def pca(df, f = None, n = 2):
    """
    pca of all features
    
    params
        df: molecular marker profiles of all cell lines tested
        f: molecular feature type, "_exp", "_lof", "_cnv", "snv", "_ddr", "__coh_pat", "__lof_pat" 
        n: number of principle components

    yields

    """
    from sklearn.decomposition import PCA
    from sklearn.preprocessing import StandardScaler

    # normalize x? for expression level must normalize by column
    #normalize by column
    #if norm:
    #    x = (x-np.mean(x, axis =0))/np.std(x, axis = 0)
        
    cell_line = df['.identifier_sample_name']
    cancer_type = df['.metadata_cancer_type']
    cancer_subtype = df['.metadata_cancer_subtype']
    data = df['data']
    new_cols = [i for i in df.columns if i.endswith(f)]
    df_new = df[new_cols]
    d=StandardScaler().fit_transform(df_new) # standarize input feature set
    pca =  PCA(n_components = n)
    pca_data = pca.fit_transform(d)
    pca_results =  pd.DataFrame(pca_data, columns= ['pca1', 'pca2'])
    pca_results['cell_line'] = cell_line
    pca_results['cancer_type'] = cancer_type
    pca_results['cancer_subtype'] = cancer_subtype
    pca_results['signature'] = f.strip('_')
    pca_results['data'] = data
    return pca_results


In [71]:
def tsne(df, f = None, n = 2):
    """
    tsne of cell line features

    params:
        df: molecular marker profiles of all cell lines tested
        f: feature type, "_exp", "_lof", "_cnv", "snv", "_ddr", "__coh_pat", "__lof_pat" 
        n: number of principle components
    """
    from sklearn.manifold import TSNE 
    from sklearn.preprocessing import StandardScaler
    cell_line = df['.identifier_sample_name']
    cancer_type = df['.metadata_cancer_type']
    cancer_subtype = df['.metadata_cancer_subtype']
    data = df['data']
    new_cols = [i for i in df.columns if i.endswith(f)]
    df_new = df[new_cols]
    d=StandardScaler().fit_transform(df_new) # standarize input feature set
    model = TSNE(n_components=n, random_state = 1)
    tsne_data = model.fit_transform(d)
    tsne_results =  pd.DataFrame(tsne_data, columns= ['tsne1', 'tsne2'])
    tsne_results['cell_line'] = cell_line
    tsne_results['cancer_type'] = cancer_type
    tsne_results['cancer_subtype'] = cancer_subtype
    tsne_results['signature'] = f.strip('_')
    tsne_results['data'] = data
    return tsne_results

In [69]:
# inport and plio transcriptomic, genetics data for all cell lines
df_mol = pd.read_csv('../feature/data/molecular_features.tsv', sep='\t')
df_mol['.identifier_sample_name'] = df_mol['.identifier_sample_name'].str.replace('ONCOLEAD_CELL_', '')
df_mol = df_mol[df_mol['.identifier_sample_name'].isin(c2t.keys())].reset_index()
df_mol['data'] = [c2d[i] for i in df_mol['.identifier_sample_name']]

In [72]:
# plot clusterin of cell lines in tsne# plot clusterin of cell lines in tsne
import plotly.express as px
import os
exp_tsne = tsne(df_mol, '_exp')
cnv_tsne = tsne(df_mol, '_cnv')
snv_tsne = tsne(df_mol, '_snv')
lof_tsne = tsne(df_mol, '_lof')
coh_pat_tsne = tsne(df_mol, '__coh_pat')
lof_pat_tsne = tsne(df_mol, '__lof_pat')
tsne_all = pd.concat([exp_tsne, cnv_tsne, snv_tsne, lof_tsne, coh_pat_tsne, lof_pat_tsne])

fig = px.scatter(tsne_all, x="tsne1", y="tsne2", color="cancer_type", facet_col='signature',facet_col_wrap=3).update_yaxes(matches=None).update_xaxes(matches=None)
fig.show()
fig.write_image("tsne.pdf")

In [80]:
tsne_all = pd.concat([exp_tsne, coh_pat_tsne])

fig = px.scatter(tsne_all, x="tsne1", y="tsne2", color="cancer_type", facet_col='signature',
facet_col_wrap=3).update_yaxes(matches=None).update_xaxes(matches=None)
fig.show()
fig.write_image("tsne_exp.pdf", scale=1, width=600, height=320)

In [79]:
len([ i for i in c2d.values() if i == 'ho1'])

16

In [74]:
fig = px.scatter(tsne_all, x="tsne1", y="tsne2", color="data", facet_col='signature',
facet_col_wrap=3).update_yaxes(matches=None).update_xaxes(matches=None)
fig.show()
fig.write_image("tsne_exp_dataset.pdf", scale=1, width=600, height=320)

,Unnamed: 0,.identifier_sample_name,.metadata_cancer_subtype,.metadata_cancer_type,AADAT_cnv,AADAT_exp,AADAT_lof,AARS2_cnv,AARS2_exp,AARS2_lof,...,ZSCAN30_cnv,ZSCAN30_exp,ZSCAN30_lof,ZSCAN4_cnv,ZSCAN4_exp,ZSCAN4_lof,ZSCAN4_snv,ZSWIM7_cnv,ZSWIM7_exp,ZSWIM7_lof
0,1,22RV1,prostate,prostate,1,6.79390,1,2,16.18963,0,...,2,23.95984,0,2,0.00000,0,0,2,27.60791,0
1,2,5637,bladder,bladder,1,24.74237,1,1,15.78652,1,...,2,6.88347,0,2,0.00000,0,0,2,18.33355,0
2,3,786O,kidney,kidney,1,2.78485,1,2,13.74952,0,...,2,10.04956,0,2,0.03996,0,0,2,21.83766,0
3,4,A204,sarcoma - rhabdomyosarcoma,sarcoma,2,2.80269,0,2,12.05400,0,...,2,14.19679,0,2,0.03615,0,0,2,24.30109,0
4,5,A2780,ovary,ovary,2,6.08043,0,2,11.43248,0,...,2,22.06030,0,2,0.15128,0,0,2,21.90708,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,88,U2OS,sarcoma - osteosarcoma,sarcoma,2,13.69688,0,2,15.13046,0,...,1,8.50146,1,2,0.22103,0,0,1,13.62605,1
88,89,U87MG,brain - GBM,brain,2,9.13501,0,2,7.34760,0,...,2,9.44710,0,2,0.00000,0,0,2,20.07623,0
89,90,UMUC3,bladder,bladder,2,10.34315,0,2,11.49480,0,...,1,4.28284,1,2,0.00000,0,0,2,21.15261,0
90,91,UO31,kidney,kidney,2,5.51459,0,2,11.44889,0,...,2,13.22354,0,2,0.00000,0,0,2,12.65176,0


In [18]:
# plot clusterin of cell lines in pca# plot clusterin of cell lines in pca
exp_pca = pca(df_mol, '_exp')
cnv_pca = pca(df_mol, '_cnv')
snv_pca = pca(df_mol, '_snv')
lof_pca = pca(df_mol, '_lof')
coh_pat_pca = pca(df_mol, '__coh_pat')
lof_pat_pca = pca(df_mol, '__lof_pat')
pca_all = pd.concat([exp_pca, cnv_pca, snv_pca, lof_pca, coh_pat_pca, lof_pat_pca])

fig = px.scatter(pca_all, x="pca1", y="pca2", color="cancer_type", facet_col='signature',
facet_col_wrap=3).update_yaxes(matches=None).update_xaxes(matches=None)
fig.show()
fig.write_image("pca.pdf")

In [22]:
c2d

{'A204': 'training',
 'A2780': 'training',
 'A549': 'training',
 'ASPC1': 'training',
 'BXPC3': 'training',
 'CACO2': 'training',
 'COLO205': 'training',
 'DLD1': 'training',
 'DU145': 'training',
 'EFO21': 'training',
 'EJ28': 'training',
 'HCT15': 'training',
 'HEPG2': 'training',
 'HS578T': 'training',
 'HS729': 'training',
 'HT1080': 'training',
 'HT29': 'training',
 'IGROV1': 'training',
 'J82': 'training',
 'JIMT1': 'training',
 'LOVO': 'training',
 'MDAMB231': 'training',
 'MDAMB435': 'training',
 'MDAMB436': 'training',
 'MDAMB468': 'training',
 'MHHES1': 'training',
 'MIAPACA2': 'training',
 'MT3': 'training',
 'NCIH292': 'training',
 'NCIH358M': 'training',
 'NCIH460': 'training',
 'NCIH82': 'training',
 'OVCAR3': 'training',
 'OVCAR4': 'training',
 'PANC1': 'training',
 'PC3': 'training',
 'PLCPRF5': 'training',
 'RD': 'training',
 'RDES': 'training',
 'SAOS2': 'training',
 'SF268': 'training',
 'SF295': 'training',
 'SKBR3': 'training',
 'SKLMS1': 'training',
 'SKMEL28': 't

In [15]:
def preprocess_monotherapy(df):
    """ generate the tissue-specificity heatmap for monothreapy drugs

    params:
    df: the orginal monotherapy
    """
    t_spec_df_all = []
    for drug in sorted(set(df['.metadata_treatment'])):
        df_sub = df.loc[df['.metadata_treatment']==drug,]
        tissue = sorted(set(df_sub['.metadata_cancer_type'].to_list()))
        a = [df_sub.loc[df_sub['.metadata_cancer_type'] ==t, '.response_aoc'].to_list() for t in tissue]
        stat, p = kruskal(*a)
        if p <= 0.05:
            print('statistically significant:', drug, '; p_val =', p)
            print("all tissue:", p)
            dunns_p = posthoc_dunn(a, p_adjust = 'bonferroni')
            dunns_p.columns = tissue
            dunns_p.index = tissue
            #print(dunns_p)
            dunns_p_df = dunns_p.unstack().to_frame().reset_index().rename(columns = {'level_0':'tissue_1','level_1':'tissue_2', 0:'p-val'})
            dunns_p_df['drug'] = drug
            t_spec_df_all.append(dunns_p_df)
        if p <= 0.001:
            print('statistically highly significant', drug,'; p_val =', p)
    t_spec_df_all = pd.concat(t_spec_df_all)
    t_spec_df_all.to_csv('./tmp/monotherapy_tissue-specific.tsv', sep='\t')
    return t_spec_df_all

In [ ]:
def preprocess_combination(df):
    '''
    '''
    df['com_treat'] = ['-'.join(sorted(list(r[['.metadata_treatment_1','.metadata_treatment_2']]))) for _,r in df.iterrows()]
    df['com_moa'] = ['-'.join(sorted(list(r[['.metadata_moa_1','.metadata_moa_2']]))) for _,r in df.iterrows()]
    t_spec_df_all = []
    # backbone ddr drugs: ATMi(M3541, M4076), ATRi(M1774,  M4344, M6620), DNAPKi(M3814)
    df_new = df = df[df['com_treat'].str.contains("M3541|M4076|M1774|M4344|M6620|M3814")]

    # compute the spearman correlation between tissue specificity (p_val) and synergy score
    p_val = []
    synergy = []
    efficacy = []

    print("Drug efficacy analysis ...")
    for com_tr in sorted(set(df_new['com_treat'])):
        df_sub = df_new.loc[df_new['com_treat']== com_tr,]
        tissue = sorted(set(df_sub['.metadata_cancer_type'].to_list()))
        a = [df_sub.loc[df_sub['.metadata_cancer_type'] ==t, '.response_aoc'].to_list() for t in tissue]
        stat, p = kruskal(*a)
        """
        if p<=0.001:
            p_val.append(2) 
        elif p<=0.05:
            p_val.append(1)
        else:
            p_val.append(0)
        #p_val.append(p) # add p-val
        synergy.append(df_sub['.response_bliss'].mean(skipna=True))
        efficacy.append(df_sub['.response_aoc'].mean(skipna=True))
        """
        #print(com_tr, p)
        if p <=0.05:
            print('statistically significant:', com_tr, '; p_val =', p)
            print('synergy_score', df_sub['.response_bliss'].mean(skipna=True))
            dunns_p = posthoc_dunn(a, p_adjust = 'bonferroni')
            dunns_p.columns = tissue
            dunns_p.index = tissue
            print(dunns_p)
            dunns_p_df = dunns_p.unstack().to_frame().reset_index().rename(columns = {'level_0':'tissue_1','level_1':'tissue_2', 0:'p-val'})
            dunns_p_df['com_treat'] = com_tr
            t_spec_df_all.append(dunns_p_df)
        if p <=0.001:
            print('statistically highly significant', com_tr,'; p_val =', p)
            print('synergy_score', df_sub['.response_bliss'].mean(skipna=True))

    t_spec_df_all = pd.concat(t_spec_df_all)
    t_spec_df_all.to_csv('./tmp/combination_tissue-specific.tsv', sep='\t')

    """
    print(p_val)
    print(synergy)
    print("synergy:")
    p_val = np.array(p_val)
    synergy = np.array(synergy)
    nas = np.logical_or(np.isnan(p_val), np.isnan(synergy))
    cor, p = spearmanr(p_val[~nas], synergy[~nas])
    print('Spearman:', cor)
    cor, p = pearsonr(p_val[~nas], synergy[~nas])
    print('Pearson:', cor)
    
    print("efficacy::")
    efficacy = np.array(efficacy)
    nas = np.logical_or(np.isnan(p_val), np.isnan(efficacy))
    cor, p = spearmanr(p_val[~nas], efficacy[~nas])
    print('Spearman:', cor)
    cor, p = pearsonr(p_val[~nas], efficacy[~nas])
    print('Pearson:', cor)
    """

    print("Drug synergy analysis ...")
    for com_tr in sorted(set(df_new['com_treat'])):
        df_sub = df_new.loc[df_new['com_treat']== com_tr,]
        tissue = sorted(set(df_sub['.metadata_cancer_type'].to_list()))
        a = [df_sub.loc[df_sub['.metadata_cancer_type'] ==t, '.response_bliss'].to_list() for t in tissue]
        stat, p = kruskal(*a)
        #print(com_tr, p)
        if p <=0.05:
            print('statistically significant:', com_tr, '; p_val =', p)
        if p <=0.001:
            print('statistically highly significant', com_tr,'; p_val =', p)




In [16]:
# import data from monoththerapy
df1= pd.read_csv('../feature/data/monotherapy_responses.tsv', sep = '\t')

In [12]:
hm_mono=preprocess_monotherapy(df1)

statistically significant: AZD4547 ; p_val = 0.04814590424501627
all tissue: 0.04814590424501627
               brain  breast  colon  hematological  liver      lung  melanoma  \
brain            1.0     1.0    1.0            1.0    1.0  1.000000  1.000000   
breast           1.0     1.0    1.0            1.0    1.0  1.000000  1.000000   
colon            1.0     1.0    1.0            1.0    1.0  1.000000  1.000000   
hematological    1.0     1.0    1.0            1.0    1.0  1.000000  1.000000   
liver            1.0     1.0    1.0            1.0    1.0  1.000000  1.000000   
lung             1.0     1.0    1.0            1.0    1.0  1.000000  1.000000   
melanoma         1.0     1.0    1.0            1.0    1.0  1.000000  1.000000   
ovary            1.0     1.0    1.0            1.0    1.0  1.000000  1.000000   
pancreas         1.0     1.0    1.0            1.0    1.0  1.000000  1.000000   
prostate         1.0     1.0    1.0            1.0    1.0  1.000000  1.000000   
sarcoma     

In [13]:
def preprocess(df):
    """ transform df to cell-line response wise matrix
    if there'are multiple responses of the same drug, 
    
    params
        df: dataframe 

    yields

    .identifier_sample_name

    """
    df['com_treat'] = ['-'.join(sorted(list(r[['.metadata_treatment_1','.metadata_treatment_2']]))) for _,r in df.iterrows()]
    df['com_moa'] = ['-'.join(sorted(list(r[['.metadata_moa_1','.metadata_moa_2']]))) for _,r in df.iterrows()]
    # only maintain DNAPKi, ATMi and ATRi combinations
    df = df[df['com_moa'].str.contains("DNAPKi|ATMi|ATRi")]
    df['ddr_type'] = df['com_moa'].str.findall("DNAPKi|ATMi|ATRi").apply('|'.join)#(lambda x: ', '.join([str(i) for i in x]))
    print(df)
    df_map = df.groupby(['data','.identifier_sample_name', '.metadata_cancer_type', 'com_treat', 'com_moa', 'ddr_type'])['.response_aoc', '.response_bliss'].mean().reset_index()
    print(df_map)
    #print(df_map_bliss)
    
    #df_new_aoc= df_map.pivot('.identifier_sample_name', 'com_treat', ".response_aoc")
    #df_new_aoc.columns.droplevel().rename(None)
    #df_new_bliss= df_map.pivot('.identifier_sample_name', 'com_treat', ".response_bliss")
    #df_new_bliss.columns.droplevel().rename(None)
    #print(df_new)
    df_new_aoc =0
    df_new_bliss =0

    return df_map, df_new_aoc, df_new_bliss

Is there a relationship between the gene expression and drug response of different tissues?

In [8]:

# final model perfromances
df_aoc = pd.read_csv('./tmp/cor_aoc.csv', header =0)
df_bliss = pd.read_csv('./tmp/cor_bliss.csv', header =0)

df_aoc['score'] = 'AoC'
df_bliss['score'] = 'Bliss'

df = pd.concat([df_aoc, df_bliss])

df

,fold,Pearson's r,features,score
0,bladder,0.637780,monotherapy+moa+molecular+target_gene+network+...,AoC
1,brain,0.832981,monotherapy+moa+molecular+target_gene+network+...,AoC
2,breast,0.812294,monotherapy+moa+molecular+target_gene+network+...,AoC
3,cervix,0.644843,monotherapy+moa+molecular+target_gene+network+...,AoC
4,colon,0.774068,monotherapy+moa+molecular+target_gene+network+...,AoC
5,hematological,0.754088,monotherapy+moa+molecular+target_gene+network+...,AoC
6,kidney,0.553038,monotherapy+moa+molecular+target_gene+network+...,AoC
7,liver,0.655052,monotherapy+moa+molecular+target_gene+network+...,AoC
8,lung,0.814705,monotherapy+moa+molecular+target_gene+network+...,AoC
9,melanoma,0.818817,monotherapy+moa+molecular+target_gene+network+...,AoC


In [33]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px


# tissue-specific performances

fig = px.bar(df, x="Pearson's r", y="fold", color = "fold", text = "Pearson's r",
        facet_col="score",
        labels=dict(fold="Tissue"))
fig.update_traces(texttemplate='%{text:.4f}', textposition='outside')
fig.update_layout(uniformtext_minsize= 10, font_family = "Arial", uniformtext_mode='hide', showlegend=False, xaxis_range = [0.5,1])
fig.show()
